In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [31]:
df = pd.read_csv("housing_final.csv", sep=";")
df = df.drop(columns=["renta_es_proyeccion", "renta_neta_anual"])

df.head()

,ccaa,provincia,anio,precio_compra_m2,precio_alquiler_m2,renta_mensual_neta,tipo_interes_hipoteca
0,Andalucia,Almería,2020,1082.6667,6.1083,809.0833,2.2333
1,Andalucia,Almería,2021,1082.4167,6.3000,841.9167,1.9904
2,Andalucia,Almería,2022,1116.5833,6.8000,883.7500,2.1038
3,Andalucia,Almería,2023,1174.7500,7.2333,961.9167,3.2812
4,Andalucia,Almería,2024,1254.8333,7.7750,1019.2206,3.4529


In [32]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [33]:
# Variable objetivo: precio de compra por m2
y = df["precio_compra_m2"]

# Features numéricas + categóricas
X = df[[
    "anio",
    "precio_alquiler_m2",
    "renta_mensual_neta",
    "tipo_interes_hipoteca",
    "ccaa",
    "provincia"
]]

# One-hot encoding para ccaa y provincia
X = pd.get_dummies(X, columns=["ccaa", "provincia"], drop_first=True)

X.shape, y.shape


((312, 73), (312,))

In [34]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

y_pred_lin = lin_reg.predict(X_test)

mse_lin = mean_squared_error(y_test, y_pred_lin)   # sin squared=
rmse_lin = np.sqrt(mse_lin)                        # raíz cuadrada -> RMSE
r2_lin   = r2_score(y_test, y_pred_lin)

print("=== LinearRegression (precio_compra_m2) ===")
print(f"RMSE test: {rmse_lin:.2f} €/m2")
print(f"R² test:   {r2_lin:.3f}")

# Opcional: validación cruzada 5-fold
scores_r2 = cross_val_score(lin_reg, X, y, cv=5, scoring="r2")
print(f"R² CV (media ± std): {scores_r2.mean():.3f} ± {scores_r2.std():.3f}")


=== LinearRegression (precio_compra_m2) ===
RMSE test: 134.20 €/m2
R² test:   0.979
R² CV (media ± std): 0.766 ± 0.068


In [35]:
coef_df_compra = pd.DataFrame({
    "Variable": X.columns,
    "Coeficiente": lin_reg.coef_
}).sort_values(by="Coeficiente", ascending=False)

coef_df_compra


,Variable,Coeficiente
53,provincia_Málaga,665.696384
23,provincia_Alicante,471.641054
27,provincia_Baleares,414.849640
6,ccaa_Baleares,414.849640
39,provincia_Girona,258.371560
...,...,...
10,ccaa_CastillaLeon,-182.444421
59,provincia_Segovia,-203.140298
60,provincia_Sevilla,-246.495112
28,provincia_Barcelona,-305.449138


RandomForestRegressor:

In [36]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("=== RandomForestRegressor (precio_compra_m2) ===")
print(f"RMSE test: {rmse_rf:.2f} €/m2")
print(f"R² test:   {r2_rf:.3f}")

# Validación cruzada 5-fold
scores_rf = cross_val_score(rf, X, y, cv=5, scoring="r2")
print(f"R² CV (media ± std): {scores_rf.mean():.3f} ± {scores_rf.std():.3f}")


=== RandomForestRegressor (precio_compra_m2) ===
RMSE test: 320.49 €/m2
R² test:   0.879
R² CV (media ± std): 0.785 ± 0.083


In [37]:
import numpy as np

rf_importances = pd.DataFrame({
    "Variable": X.columns,
    "Importancia": rf.feature_importances_
}).sort_values(by="Importancia", ascending=False)

rf_importances


,Variable,Importancia
1,precio_alquiler_m2,0.902247
0,anio,0.020049
2,renta_mensual_neta,0.016618
66,provincia_Valencia,0.005640
11,ccaa_Cataluña,0.005225
...,...,...
52,provincia_Murcia,0.000044
26,provincia_Badajoz,0.000033
31,provincia_Castellón,0.000026
35,provincia_Cáceres,0.000025


GradientBoostingRegressor:

In [38]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=3,
    subsample=0.8,
    random_state=42
)

gbr.fit(X_train, y_train)

y_pred_gbr = gbr.predict(X_test)

mse_gbr = mean_squared_error(y_test, y_pred_gbr)
rmse_gbr = np.sqrt(mse_gbr)
r2_gbr = r2_score(y_test, y_pred_gbr)

print("\n=== GradientBoostingRegressor (precio_compra_m2) ===")
print(f"RMSE test: {rmse_gbr:.2f} €/m2")
print(f"R² test:   {r2_gbr:.3f}")

# Validación cruzada 5-fold
scores_gbr = cross_val_score(gbr, X, y, cv=5, scoring="r2")
print(f"R² CV (media ± std): {scores_gbr.mean():.3f} ± {scores_gbr.std():.3f}")



=== GradientBoostingRegressor (precio_compra_m2) ===
RMSE test: 240.96 €/m2
R² test:   0.932
R² CV (media ± std): 0.810 ± 0.072


In [39]:
gbr_importances = pd.DataFrame({
    "Variable": X.columns,
    "Importancia": gbr.feature_importances_
}).sort_values(by="Importancia", ascending=False)

gbr_importances


,Variable,Importancia
1,precio_alquiler_m2,0.873395
0,anio,0.017284
2,renta_mensual_neta,0.016249
66,provincia_Valencia,0.014055
53,provincia_Málaga,0.010700
...,...,...
31,provincia_Castellón,0.000000
52,provincia_Murcia,0.000000
47,provincia_León,0.000000
61,provincia_Soria,0.000000


Ahora, con el precio del alquiler

In [40]:
# Variable objetivo: precio de compra por m2
y = df["precio_alquiler_m2"]

# Features numéricas + categóricas
X = df[[
    "anio",
    "precio_compra_m2",
    "renta_mensual_neta",
    "tipo_interes_hipoteca",
    "ccaa",
    "provincia"
]]

# One-hot encoding para ccaa y provincia
X = pd.get_dummies(X, columns=["ccaa", "provincia"], drop_first=True)

X.shape, y.shape

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

y_pred_lin = lin_reg.predict(X_test)

mse_lin = mean_squared_error(y_test, y_pred_lin)   # sin squared=
rmse_lin = np.sqrt(mse_lin)                        # raíz cuadrada -> RMSE
r2_lin   = r2_score(y_test, y_pred_lin)

print("=== LinearRegression (precio_alquiler_m2) ===")
print(f"RMSE test: {rmse_lin:.2f} €/m2")
print(f"R² test:   {r2_lin:.3f}")

# Opcional: validación cruzada 5-fold
scores_r2 = cross_val_score(lin_reg, X, y, cv=5, scoring="r2")
print(f"R² CV (media ± std): {scores_r2.mean():.3f} ± {scores_r2.std():.3f}")

=== LinearRegression (precio_alquiler_m2) ===
RMSE test: 0.50 €/m2
R² test:   0.980
R² CV (media ± std): 0.871 ± 0.048


In [41]:
coef_df_alquiler = pd.DataFrame({
    "Variable": X.columns,
    "Coeficiente": lin_reg.coef_
}).sort_values(by="Coeficiente", ascending=False)

coef_df_alquiler

,Variable,Coeficiente
28,provincia_Barcelona,2.699960
66,provincia_Valencia,1.821038
60,provincia_Sevilla,1.396169
59,provincia_Segovia,1.127483
46,provincia_Las Palmas,1.087777
...,...,...
55,provincia_Ourense,-0.645436
44,provincia_Jaén,-0.821126
69,provincia_Zamora,-0.837909
62,provincia_Tarragona,-0.951487


RandomForestRegressor:

In [42]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("=== RandomForestRegressor (precio_alquiler_m2) ===")
print(f"RMSE test: {rmse_rf:.2f} €/m2")
print(f"R² test:   {r2_rf:.3f}")

# Validación cruzada 5-fold
scores_rf = cross_val_score(rf, X, y, cv=5, scoring="r2")
print(f"R² CV (media ± std): {scores_rf.mean():.3f} ± {scores_rf.std():.3f}")


=== RandomForestRegressor (precio_alquiler_m2) ===
RMSE test: 0.92 €/m2
R² test:   0.933
R² CV (media ± std): 0.853 ± 0.051


In [44]:
rf_importances_a = pd.DataFrame({
    "Variable": X.columns,
    "Importancia": rf.feature_importances_
}).sort_values(by="Importancia", ascending=False)

rf_importances_a


,Variable,Importancia
1,precio_compra_m2,0.904382
0,anio,0.028898
2,renta_mensual_neta,0.024442
66,provincia_Valencia,0.006669
3,tipo_interes_hipoteca,0.006415
...,...,...
33,provincia_Ciudad Real,0.000034
34,provincia_Cuenca,0.000029
35,provincia_Cáceres,0.000028
64,provincia_Teruel,0.000026


GradientBoostingRegressor:

In [45]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=3,
    subsample=0.8,
    random_state=42
)

gbr.fit(X_train, y_train)

y_pred_gbr = gbr.predict(X_test)

mse_gbr = mean_squared_error(y_test, y_pred_gbr)
rmse_gbr = np.sqrt(mse_gbr)
r2_gbr = r2_score(y_test, y_pred_gbr)

print("\n=== GradientBoostingRegressor (precio_alquiler_m2) ===")
print(f"RMSE test: {rmse_gbr:.2f} €/m2")
print(f"R² test:   {r2_gbr:.3f}")

# Validación cruzada 5-fold
scores_gbr = cross_val_score(gbr, X, y, cv=5, scoring="r2")
print(f"R² CV (media ± std): {scores_gbr.mean():.3f} ± {scores_gbr.std():.3f}")



=== GradientBoostingRegressor (precio_alquiler_m2) ===
RMSE test: 0.65 €/m2
R² test:   0.967
R² CV (media ± std): 0.866 ± 0.065


In [46]:
gbr_importances_a = pd.DataFrame({
    "Variable": X.columns,
    "Importancia": gbr.feature_importances_
}).sort_values(by="Importancia", ascending=False)

gbr_importances_a


,Variable,Importancia
1,precio_compra_m2,0.873131
0,anio,0.034392
2,renta_mensual_neta,0.027709
66,provincia_Valencia,0.011208
28,provincia_Barcelona,0.010092
...,...,...
47,provincia_León,0.000000
64,provincia_Teruel,0.000000
63,provincia_Tenerife,0.000000
61,provincia_Soria,0.000000


### Importancia de las variables

A partir de la regresión lineal y de los modelos de RandomForest y Gradient Boosting, podemos extraer varias conclusiones comunes:

- Para explicar el **precio de compra por m²**, la variable más relevante es el **precio de alquiler por m²** en la misma provincia y año. A mayor alquiler, mayor precio de compra. En segundo lugar aparecen el **año**, la **renta neta mensual** y algunas provincias concretas (por ejemplo Málaga, Alicante o Baleares), que captan diferencias estructurales entre mercados inmobiliarios.
- Para explicar el **precio de alquiler por m²**, ocurre lo contrario: la variable con mayor importancia es el **precio de compra por m²**, seguida de la **renta neta mensual**, el año y ciertas provincias (como Barcelona o Valencia). Es decir, los mercados de compra y alquiler están fuertemente acoplados.
- Las variables de localización (CCAA y provincia) tienen un peso claro, pero el efecto principal está en el nivel de precios (compra ↔ alquiler) y en la renta disponible. Esto sugiere que el comportamiento medio del mercado se puede capturar bien con un conjunto relativamente reducido de variables económicas y geográficas.

En resumen, los tres modelos coinciden en que los precios de compra y alquiler están muy correlacionados entre sí, y que la renta de los hogares y la localización geográfica actúan como moduladores del nivel de precios.


## Conclusiones del análisis exploratorio

- Entre 2020 y 2025 los **precios medios de compra por m²** muestran una tendencia claramente creciente en el conjunto de España, mientras que los **precios de alquiler** también suben, pero con una pendiente menor.
- Existe una **fuerte relación entre compra y alquiler**: la correlación entre `precio_compra_m2` y `precio_alquiler_m2` es muy alta (~0.93), lo que indica que los mercados de compra y de alquiler se mueven de forma coordinada.
- Las variables de **renta** (`renta_neta_anual`, `renta_mensual_neta`) están moderadamente correlacionadas con los precios (correlaciones alrededor de 0.5–0.6): los hogares con mayor renta tienden a vivir en zonas más caras, pero la renta no lo explica todo.
- El **tipo de interés hipotecario** tiene una relación más débil con el precio en este rango temporal (correlaciones en torno a 0.1–0.3), probablemente porque el efecto del tipo se transmite con retraso y está mezclado con otros factores macro.
- La **localización geográfica** es clave: los boxplots por CCAA muestran grandes diferencias estructurales. Comunidades como Baleares, Madrid o Cataluña concentran los precios más altos, mientras que otras como Extremadura o Castilla-La Mancha se sitúan en la parte baja de la distribución.

---

## Conclusiones de los modelos explicativos

- Para explicar el **precio de compra por m²**, la regresión lineal muestra que:
  - El predictor más importante es el **precio de alquiler por m²**: a mayor alquiler, mayor precio de compra.
  - La **renta mensual neta** y el **año** añaden capacidad explicativa, pero con un efecto marginal menor que el alquiler.
  - Algunas provincias (por ejemplo Málaga, Alicante o Baleares) presentan coeficientes muy positivos, indicando que, a igualdad de renta y alquiler, tienen un nivel estructural de precios más alto que la provincia de referencia.

- Para explicar el **precio de alquiler por m²** ocurre algo simétrico:
  - El **precio de compra por m²** es el driver principal del alquiler.
  - La **renta mensual** y el **año** ajustan diferencias de poder adquisitivo y tendencia temporal.
  - Provincias como Barcelona o Valencia aparecen con coeficientes positivos, asociadas a niveles de alquiler superiores a la media.

- En ambos casos, los modelos de **RandomForest** y **Gradient Boosting** coinciden con la regresión lineal en las variables más relevantes (compra ↔ alquiler, renta, año, localización), lo que da robustez a las conclusiones.

---

## Conclusiones de los modelos predictivos

- Tanto para compra como para alquiler se han probado tres familias de modelos:
  - **Regresión lineal**
  - **RandomForestRegressor**
  - **GradientBoostingRegressor**

- Resultados para **precio de compra por m²**:
  - La **regresión lineal** consigue un R² de test cercano a **0.98** y un RMSE de ~**130 €/m²**, con un R² de validación cruzada alrededor de **0.77**.
  - **RandomForest** y **Gradient Boosting** obtienen R² entre **0.88 y 0.93**, con RMSE claramente mayores.
  - En este problema, los modelos de árbol no mejoran a la regresión lineal, por lo que el modelo lineal es una buena elección tanto por rendimiento como por interpretabilidad.

- Resultados para **precio de alquiler por m²**:
  - La **regresión lineal** vuelve a ser muy competitiva, con un R² de test ~**0.98** y un RMSE de ~**0.5 €/m²**.
  - **RandomForest** y **Gradient Boosting** ofrecen R² en el rango **0.93–0.97** y RMSE algo superiores, confirmando los mismos patrones pero sin aportar una mejora clara de error.
  - De nuevo, el modelo lineal ofrece el mejor equilibrio entre precisión y simplicidad.

- En resumen, el dataset es relativamente “amable”: la relación entre variables es bastante lineal y la varianza se explica muy bien con pocas features (`precio_*`, renta, año y localización). Por eso, los modelos lineales funcionan incluso mejor que los métodos de ensemble más complejos.

---

## Líneas futuras de mejora

- Incorporar variables adicionales (por ejemplo, tasa de paro, población, stock de vivienda nueva, coste de la financiación a más largo plazo…) podría capturar mejor los factores macro y reducir aún más el error.
- Probar modelos espaciales (efectos de vecindario entre provincias o CCAA) o modelos de series temporales por provincia para analizar dinámicas locales.
- Evaluar la estabilidad temporal entrenando el modelo sólo con años anteriores y probando en el último año disponible, para aproximar un escenario de predicción “real” a futuro.


Lineal (coef_df_compra)

Estás usando one-hot para CCAA/provincia, así que:

Los coeficientes de provincias/CCAA se interpretan relativos a la categoría de referencia (la CCAA/provincia que se ha quedado fuera del one-hot).

Lo que se ve en la tabla:

Coeficientes muy positivos:

provincia_Málaga, provincia_Alicante, ccaa_Baleares, provincia_Girona, etc.
→ Provincias donde, manteniendo constantes el resto de variables, el precio de compra por m² es considerablemente más alto que en la categoría base.

Coeficientes muy negativos:

ccaa_CastillaLeon, provincia_Segovia, provincia_Sevilla, provincia_Valencia, etc. (según tu tabla concreta)
→ Provincias / CCAA donde el precio de compra tiende a ser más bajo que la base.

El coeficiente de precio_alquiler_m2 es grande y positivo → a mayor alquiler, mayor precio de compra.

renta_mensual_neta tiene coeficiente pequeño → su efecto marginal es mucho menor que el de la localización o el alquiler.

RandomForest / GradientBoosting (rf_importances, gbr_importances)

Ambas técnicas coinciden bastante:

Variable más importante: precio_alquiler_m2.

Después aparecen anio y renta_mensual_neta con importancia moderada.

Algunas provincias concretas (p.ej. provincia_Valencia, ccaa_Cataluña, provincia_Málaga…) también pesan, pero menos que el alquiler.

Interpretación:

El mejor predictor del precio de compra no es tanto la renta o el tipo de interés, sino el propio precio de alquiler en esa provincia/año.

La renta media y el año aportan ajuste fino (captan crecimiento temporal y nivel socioeconómico), y las provincias captan diferencias estructurales de mercado.

Lineal (coef_df_alquiler)

De nuevo, coeficientes relativos a la provincia/CCAA de referencia.

En tu tabla se ve:

Top positivos: provincia_Barcelona, provincia_Valencia, provincia_Sevilla, provincia_Segovia, provincia_Las Palmas, etc.
→ Zonas donde el alquiler por m² es claramente superior a la base.

Top negativos: provincia_Ourense, provincia_Jaén, provincia_Zamora, provincia_Tarragona, provincia_Alicante, etc.
→ Zonas con alquiler por m² inferior a la base.

El coeficiente de precio_compra_m2 es positivo y grande → donde la vivienda se compra cara, también se alquila caro.

renta_mensual_neta vuelve a aportar pero con efecto relativamente pequeño.

RandomForest / GradientBoosting (rf_importances_a, gbr_importances_a)

Los dos modelos están muy de acuerdo:

Variable más importante: precio_compra_m2 (importancia altísima).

Luego renta_mensual_neta y anio.

Algunas provincias destacan (provincia_Valencia, provincia_Barcelona…), pero con importancia mucho menor que el precio de compra.

Interpretación:

Para explicar / predecir el alquiler, el precio de compra es el driver principal.

La renta y el año ajustan diferencias de poder adquisitivo y tendencia temporal, y la provincia capta las diferencias estructurales de mercado.